#### Setting up the Request

In [ ]:
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import json
import pandas as pd
import time



API_KEY='api_key'
USER_AGENT='name'
headers={'user-agent': USER_AGENT}
payload={'api_key':'api_key' ,
        'method': 'chart.gettopartists',
        'format': 'json'}
r=requests.get('http://ws.audioscrobbler.com/2.0/', headers = headers , params = payload)
r.status_code


#### Testing the status of the request

In [ ]:
test=requests.get('http://ws.audioscrobbler.com/2.0/', headers={'user-agent':'name'} , 
                  params = {'api_key':'api_key' , 'method': 'chart.gettopartists' , 'format': 'json'})
test.status_code

#### Checking json's format

In [ ]:
#Checing the format to make the call

def jprint(obj):
    text = json.dumps(obj, sort_keys = True, indent = 4)
    print(text)

jprint(r.json())

#### Scrapping the names of all the Artists

In [ ]:
results = []
page = 1
total_pages = 99999 

while (page <= total_pages):
    payload = {
        #the parameters are from the:
        ##https://www.last.fm/api/show/chart.getTopArtists
        'api_key':'api_key',
        'format':'json',
        'method': 'chart.gettopartists',
        'limit': 500,
        'page': page
    }
    
    print(page,' out of ', total_pages)

    #Making the call
    result = requests.get('http://ws.audioscrobbler.com/2.0/'
                            , headers = {'user-agent': 'fil_theo'}
                            , params = payload)
    
    # If sth goes wrong
    if result.status_code != 200:
        print(response.text)
        break
        
    page = int(result.json()['artists']['@attr']['page'])
    total_pages = int(result.json()['artists']['@attr']['totalPages'])
    results.append(result)
    
    #This stops the error of making consecutive requests
    if not getattr(result, 'from_cache', False):
        time.sleep(0.25)
        
    #next page
    page =page + 1

#### Merging the pages and creating the dataframe

In [ ]:
pages = [pd.DataFrame(r.json()['artists']['artist']) for r in responses]
artists = pd.concat(pages)
artists

#### Checking the format for the tags for every Artist

In [ ]:

r = lastfm_get({
    'method': 'artist.getTopTags',
    'artist':  'Lana Del Rey'
})
jprint(r.json())

#### Scrapping the top 4 tags for every Artist

In [ ]:
#Function to get tags for every artist
def get_tags(artist):
    payload={'method': 'artist.getTopTags',
        'artist':  artist,
        'api_key':'api_key',
        'format':'json',}
    result = requests.get('http://ws.audioscrobbler.com/2.0/'
                            , headers={'user-agent': 'fil_theo'}
                            ,params=payload)
    

    # Stop for error
    if result.status_code != 200:
        return None

    #Get the top 4 tags
    tags = [t['name'] for t in result.json()['toptags']['tag'][:4]]
    tags_str = ', '.join(tags)

    #Responses error
    if not getattr(response, 'from_cache', False):
        time.sleep(0.25)
    return tags_str

#Making the call for every every artist:
artists['tags'] = artists['name'].progress_apply(get_tags)

#### Saving and Downloading

In [ ]:
import os

outname = 'artists.csv'

outdir = 'D:/uni/'
if not os.path.exists(outdir):
    os.mkdir(outdir)
fullname = os.path.join(outdir, outname) 
artists.to_csv(fullname)